In [7]:
from asitop.utils import parse_gpu_metrics, parse_powermetrics, run_powermetrics_process, parse_cpu_metrics

In [8]:
cpu_metrics_dict, gpu_metrics_dict, thermal_pressure, bandwidth_metrics, timestamp = parse_powermetrics('powermetrics.txt', timecode='')
gpu_metrics_dict

{'freq_MHz': 388, 'active': 6}

In [9]:
import plistlib
import pandas as pd

with open('powermetrics.txt', 'rb') as fp:
    data = fp.read()

metrics_list = []    
for d in data.split(b'\x00'):
    d = plistlib.loads(d)
    
    cpu_m = parse_cpu_metrics(d)

    metrics_list.append({
        "GPU_freq_MHz": int(d['gpu']["freq_hz"]),
        "GPU_active": int((1 - d['gpu']["idle_ratio"])*100),
        **cpu_m
        
    })

orig_df = pd.DataFrame(metrics_list)
orig_df.head()

,GPU_freq_MHz,GPU_active,E-Cluster_freq_Mhz,E-Cluster_active,E-Cluster0_freq_Mhz,E-Cluster0_active,E-Cluster1_freq_Mhz,E-Cluster1_active,P0-Cluster_freq_Mhz,P0-Cluster_active,...,P-Cluster9_freq_Mhz,P-Cluster9_active,e_core,p_core,P-Cluster_active,P-Cluster_freq_Mhz,ane_W,cpu_W,gpu_W,package_W
0,388,4,1305,51,1330,42,1310,41,1164,30,...,0,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",15,1164,0.0,0.466,0.032,0.494351
1,434,15,1529,65,1677,55,1669,54,1821,49,...,2580,7,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",36,1821,0.0,3.107,0.156,3.166130
2,388,6,1493,65,1528,53,1526,53,1533,41,...,1918,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",29,1533,0.0,0.920,0.042,0.936622
3,390,5,1561,64,1569,53,1569,50,1611,33,...,1984,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",23,1611,0.0,0.744,0.053,0.774890
4,388,5,1554,60,1569,49,1576,48,1157,28,...,1268,0,"[0, 1]","[2, 3, 4, 5, 6, 7, 8, 9]",16,1157,0.0,0.522,0.031,0.541158


In [10]:
df.columns

Index(['index', 'GPU_freq_MHz', 'GPU_active', 'E-Cluster_freq_Mhz',
       'E-Cluster_active', 'E-Cluster0_freq_Mhz', 'E-Cluster0_active',
       'E-Cluster1_freq_Mhz', 'E-Cluster1_active', 'P0-Cluster_freq_Mhz',
       'P0-Cluster_active', 'P-Cluster2_freq_Mhz', 'P-Cluster2_active',
       'P-Cluster3_freq_Mhz', 'P-Cluster3_active', 'P-Cluster4_freq_Mhz',
       'P-Cluster4_active', 'P-Cluster5_freq_Mhz', 'P-Cluster5_active',
       'P1-Cluster_freq_Mhz', 'P1-Cluster_active', 'P-Cluster6_freq_Mhz',
       'P-Cluster6_active', 'P-Cluster7_freq_Mhz', 'P-Cluster7_active',
       'P-Cluster8_freq_Mhz', 'P-Cluster8_active', 'P-Cluster9_freq_Mhz',
       'P-Cluster9_active', 'e_core', 'p_core', 'P-Cluster_active',
       'P-Cluster_freq_Mhz', 'ane_W', 'cpu_W', 'gpu_W', 'package_W'],
      dtype='object')

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


df = orig_df[5:].copy()
df.reset_index(inplace=True)

# Create a figure with two subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Add traces
fig.add_trace(go.Scatter(x=df.index, y=df['P-Cluster_active'], mode='lines', name='CPU P-Cluster %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['E-Cluster_active'], mode='lines', name='CPU E-Cluster %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.GPU_active, mode='lines', name='GPU Active (time) %'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df.GPU_freq_MHz, mode='lines', name='GPU freq (MHz)'), row=2, col=1)

# Update layout if needed (titles, axis labels etc.)
# fig.update_layout(height=400, width=800, title_text="CPU/GPU usage for CPU-based torch model inference")
fig.update_layout(height=400, width=800, title_text="CPU/GPU usage for PyTroch MPS model inference")
fig.update_xaxes(title_text="Time (seconds)", row=2, col=1)  # Update only the bottom x-axis

fig.update_yaxes(range=[0, 100], row=1, col=1)




fig.add_annotation(
        x=df.GPU_active.idxmax(),
        y=df.GPU_active.max(),
        xref="x",
        yref="y",
        text=f"Max of GPU Active  = {df.GPU_active.max()}%",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="green",
        opacity=0.6,
        row=1, col=1
    )

fig.add_annotation(
        x=df.GPU_freq_MHz.idxmax(),
        y=df.GPU_freq_MHz.max(),
        xref="x",
        yref="y",
        text=f"Max of GPU frequency  = {df.GPU_freq_MHz.max()} MHz",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="magenta",
        opacity=0.6,
        row=2, col=1
    )

# Show the figure
fig.show()



# Bar chart for time

In [38]:
import plotly.graph_objects as go

devices = ["PyTorch CPU", "PyTorch MPS", "MLX"]
time_consumed = [79, 18, 7]  # measured

# Creating a bar chart
fig = go.Figure(data=[go.Bar(
    y=devices, x=time_consumed, orientation='h',
    textposition='inside',
    text=[f'{t}s <br> {round(200/t,1)} token/s' for t in time_consumed]
)])

# Adding title and labels
fig.update_layout(
    title="Benchmarking Speed",
    xaxis_title="Time Consumed (seconds) for 200 tokens",
    yaxis_title="",
    height=400, width=1000,
)


# Display the figure
fig.show()
